In [2]:
import torch
import torch.nn as nn
import numpy as np 
import os
import imageio
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from tqdm import tqdm

from dataset import get_rays 
from rendering import Rendering
from model import Voxels, NeRF
from ml_helpers import training

In [3]:
import mcubes
import trimesh

In [4]:
device='cuda'

tn=8.
tf=12.
model=torch.load('model_nerf')

C:\Users\elvee\AppData\Local\Temp\ipykernel_14532\3708406446.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load('model_nerf')


In [5]:
N=100
scale=1.5

x=torch.linspace(-scale,scale,N)
y=torch.linspace(-scale,scale,N)
z=torch.linspace(-scale,scale,N)

x, y, z=torch.meshgrid((x,y,z))

xyz=torch.cat((x.reshape(-1,1),
               y.reshape(-1,1),
               z.reshape(-1,1)),dim=1)

c:\Users\elvee\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
with torch.no_grad():
    _,density=model.forward(xyz.to(device), torch.zeros_like(xyz).to(device)) 

In [7]:
density=density.cpu().numpy().reshape(N,N,N)


In [ ]:
vertices,triangles=mcubes.marching_cubes(density,np.mean(density))


In [12]:
mesh=trimesh.Trimesh(vertices,triangles)
mesh.show()